In [ ]:
import rasterio
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from rasterimport geopandas as gpdio.transform import from_origin
import geopandas as gpd

# Uydu görüntüsünü yükleyin (örneğin, "image.tif" gibi).
stacked_image_path  = "path/to/your/image.tif"

In [ ]:
# Noktaları .shp formatında yükleyin.
points_path = "path/to/your/points.shp"
points_data = gpd.read_file(points_path)


# Noktaların koordinatlarını alın.
coordinates = list(zip(points_data.geometry.x, points_data.geometry.y))

# Noktaların sınıf etiketlerini alın
class_labels = list(points_data['Sinif'])  # 'Sinif' sütun adını güncelleyin


# Noktaların piksel değerlerini alın.
pixel_values = []

# Stack'lenmiş görüntüyü açın
with rasterio.open(stacked_image_path) as stacked_image:
    # Görüntü içindeki her bant için
    for band_num in range(1, 61):  # Toplam 60 bant varsa
        # Piksel değerlerini tutacak bir liste oluşturun
        band_pixel_values = []
        for coord in coordinates:
            row, col = rasterio.transform.rowcol(stacked_image.transform, coord[0], coord[1])
            pixel_value = stacked_image.read(band_num, window=((row, row+1), (col, col+1)))
            band_pixel_values.append(pixel_value)
        pixel_values.append(band_pixel_values)

In [ ]:
# Eğitim verileri için özellik matrisi (X_train) ve etiketler (y_train) oluşturun.
X_train = np.array(pixel_values)
y_train = np.array(class_labels)  # Etiketlerin bulunduğu sütunu belirleyin.

In [ ]:
# Random Forest sınıflandırıcı modelini oluşturun ve eğitin.
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

In [ ]:
# Test verileri de uygun bir şekilde hazırlanmalıdır, benzer şekilde label sütunlarını kullanarak eğitim verileri hazırladığınız gibi yapabilirsiniz.
# Test verileri için özellik matrisi (X_test) oluşturun.
# Örnek olarak, uygun test noktalarının piksel değerlerini alabilirsiniz:
test_points_path = "path/to/your/test_points.shp"
test_points_data = gpd.read_file(test_points_path)

test_coordinates = list(zip(test_points_data.geometry.x, test_points_data.geometry.y))
test_pixel_values = []

for coord in test_coordinates:
    row, col = rasterio.transform.rowcol(satellite_image.transform, coord[0], coord[1])
    test_pixel_value = satellite_image.read(indexes=[1, 2, 3, 4], window=((row, row+1), (col, col+1)))
    test_pixel_values.append(test_pixel_value)

X_test = np.array(test_pixel_values)
y_test = test_points_data['Crop_Type']  # Test verilerinize uygun şekilde etiketleri de belirleyin.

# Modeli değerlendirme:
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)

print("Doğruluk: ", accuracy)
print("Sınıflandırma Raporu:")
print(classification_report_result)

In [ ]:
# Sınıflandırılmış görüntüyü oluşturun.
y_pred_image = np.zeros_like(red_band, dtype=np.uint8)  # Red_band ile aynı boyutta bir sınıflandırılmış görüntü oluşturun.

# Sınıflandırılmış değerleri sınıflandırıcı sonuçlarına göre atayın.
for row in range(y_pred_image.shape[0]):
    for col in range(y_pred_image.shape[1]):
        y_pred_image[row, col] = y_pred[row * y_pred_image.shape[1] + col]

# Sınıflandırılmış görüntüyü kaydedin.
output_path = "path/to/your/classified_image.tif"  # Kaydedilecek dosyanın yolu ve adını belirleyin.
transform = from_origin(satellite_image.bounds.left, satellite_image.bounds.top, satellite_image.res[0], satellite_image.res[1])

with rasterio.open(output_path, 'w', driver='GTiff', height=y_pred_image.shape[0], width=y_pred_image.shape[1],
                   count=1, dtype=str(y_pred_image.dtype), crs=satellite_image.crs, transform=transform) as dst:
    dst.write(y_pred_image, indexes=1)